# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion the project. 


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
except Exception as e:
    print(e)
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""create keyspace if not exists udacity with replication =
                    {'class':'SimpleStrategy','replication_factor':1}""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

Here (session_id,item_number_session) were chosen as primary key as session_id is duplicated and also because they are used in the query 

In [27]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query="create table if not exists song_playlist_library"
query=query+"(session_id int,\
              item_number_session int ,\
              artist text ,\
              song_title text, \
              length_song float,\
              primary key((session_id,item_number_session)))"
session.execute(query)             

In [28]:
file = 'event_datafile_new.csv'
#artist = line[0]
#first_name_user = line[1]
#gender = line[2]
#item_number_session = line[3]
#last_name_user = line[4]
#length_song = line[5]
#level = line[6]
#location = line[7]
#session_id = line[8]
#song_title = line[9]
#user_id = line[10]
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_playlist_library (session_id ,\
                                            item_number_session  ,\
                                            artist  ,\
                                            song_title , \
                                            length_song  )"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

The result shows only one result  

In [29]:
query="select artist ,length_song ,song_title \
       from song_playlist_library \
       where session_id = 338 and item_number_session = 4"
rows=session.execute(query)
for row in rows :
    print(row.artist ,row.length_song ,row.song_title)

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


Here (user_id,session_id) are used as a composite key and item_number_session is used as clustering column as it is required to sort depending on it 

In [30]:
##  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query="create table if not exists artist_user_library"
query=query+"(user_id int,\
              session_id int,\
              item_number_session int,\
              artist text,\
              first_name_user text,\
              last_name_user text,\
              song_title text,\
              primary key ((user_id,session_id),item_number_session))"
session.execute(query)   


In [31]:
file = 'event_datafile_new.csv'
#artist = line[0]
#first_name_user = line[1]
#gender = line[2]
#item_number_session = line[3]
#last_name_user = line[4]
#length_song = line[5]
#level = line[6]
#location = line[7]
#session_id = line[8]
#song_title = line[9]
#user_id = line[10]
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into artist_user_library (user_id ,\
                                                  session_id ,\
                                                  item_number_session ,\
                                                  artist,first_name_user ,\
                                                  last_name_user ,\
                                                  song_title )"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[4],line[9]))

There are 4 results for this query for the same user

In [32]:
query="select artist ,song_title,first_name_user,last_name_user \
       from artist_user_library \
       where user_id = 10 and session_id = 182\
       order by item_number_session "
rows=session.execute(query)
for row in rows :
    print(row.artist ,row.song_title,row.first_name_user,row.last_name_user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


Here we used (song_title,user_id) as composite key as song title can be repeated and also can restrict the table to store only one record for a song listened to by multiple users with the same name 

In [22]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query="create table if not exists user_library"
query=query+"(song_title text,\
              user_id int ,\
              first_name_user text,\
              last_name_user text,\
              primary key ((song_title),user_id))"
session.execute(query)                                

In [23]:
file = 'event_datafile_new.csv'
#artist = line[0]
#first_name_user = line[1]
#gender = line[2]
#item_number_session = line[3]
#last_name_user = line[4]
#length_song = line[5]
#level = line[6]
#location = line[7]
#session_id = line[8]
#song_title = line[9]
#user_id = line[10]
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into user_library (song_title,\
                                           user_id,\
                                           first_name_user ,\
                                           last_name_user )"
        query = query + "values(%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [24]:
query="select first_name_user,last_name_user \
       from user_library \
       where song_title = 'All Hands Against His Own' "
rows=session.execute(query)
for row in rows :
    print(row.first_name_user ,row.last_name_user  )

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [34]:
query="drop table song_playlist_library"
session.execute(query)

In [35]:
query="drop table artist_user_library"
session.execute(query)

In [36]:
query="drop table user_library"
session.execute(query)

### Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()